<a href="https://colab.research.google.com/github/johanesPao/tensorflow_developer_certificate/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problem (seq2seq).

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-561681ae-2d2f-d9e2-abfd-e0d2f2a3255d)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, plot_loss_curves, compare_historys

# Install, import and login to wandb for model monitoring
!pip install wandb
import wandb as wb
wb.login(key='924d78a46727fe1fb5374706bf1b8a158fe73971')

--2022-06-23 02:27:35--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-06-23 02:27:36 (110 MB/s) - ‘helper_functions.py’ saved [10246/10246]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.5 MB/s 
     |████████████████████████████████| 181 kB 55.9 MB/s 
     |████████████████████████████████| 145 kB 29.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

See the original source here: https://www.kaggle.com/competitions/nlp-getting-started/

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data('nlp_getting_started.zip')

--2022-06-23 02:27:54--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 142.250.1.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-06-23 02:27:54 (161 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use Python: https://realpython.com/read-write-files-python/

But I prefer to get visual straight away.

So another way to do this is to use pandas...

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples

for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}', '(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n{text}\n')
  print('---\n')

Target: 0 (not real disaster)
Text:
Illusoria Icarus nowplaying check out http://t.co/E2WgREIcmZ

---

Target: 0 (not real disaster)
Text:
Why put out a fire when it's still burning.

---

Target: 0 (not real disaster)
Text:
@Squibby_ I am an eyewitness HA

---

Target: 1 (real disaster)
Text:
(#LosDelSonido) Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Ma...  (#IvanBerroa)

---

Target: 0 (not real disaster)
Text:
Bluedio Turbine Hurricane H Bluetooth 4.1 Wireless Stereo Headphones Headset BLK - Full reÛ_ http://t.co/jans3Fd4lf http://t.co/2SdMichb2Z

---



### Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation split
                                                                            random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embedding - creaet a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [14]:
train_sentences[:15]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt',
       'Lava Dragon Breeder! I just bred a 

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=max_vocab_length, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?
                                    output_mode='int', # how to map tokens to number
                                    output_sequence_length=max_length, # how long do you want your sequences to be
                                    pad_to_max_tokens=True)

In [16]:
len(train_sentences[0].split())

7

In [17]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [18]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [19]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f'Original text:\n{random_sentence}\
      \n\nVectorized version:')
text_vectorizer([random_sentence])

Original text:
Panic attacks are the worst ????      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 297, 1553,   22,    2,  971,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [21]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f'Number of words in vocab: {len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating and Embedding using an Embedding Layer

To make our embedding, we're going to use TensorFlow's embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [22]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape
                             input_length=max_length # how long is each input
                             )

embedding

In [23]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence}\
      \n\nEmbedded version:')

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 9000 Photographs from 1800's British Mandate of Palestine - with no trace of 'Palestinians' http://t.co/X8i0mHYRmN      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00321428,  0.02496641, -0.00829329, ...,  0.01385087,
         -0.0310065 , -0.04316831],
        [-0.00348393, -0.03371785, -0.00593475, ...,  0.00100155,
         -0.01217087, -0.00606268],
        [-0.03514956, -0.0144629 ,  0.00423983, ..., -0.02655351,
          0.02000954, -0.00561972],
        ...,
        [-0.01984397, -0.0080728 , -0.03391819, ..., -0.03334771,
         -0.0341743 ,  0.01062755],
        [-0.0433182 , -0.04442134, -0.03183293, ...,  0.04657563,
          0.04864432,  0.01968605],
        [-0.01778482,  0.03708119,  0.03235256, ...,  0.02948297,
         -0.01010273,  0.04215525]]], dtype=float32)>

In [24]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00321428,  0.02496641, -0.00829329,  0.0103545 , -0.03193368,
         0.03501331,  0.03667385,  0.03720793,  0.04099888,  0.01823156,
         0.0127519 , -0.00841822,  0.01195897,  0.00149615,  0.04553875,
        -0.04939414, -0.02094234, -0.0039302 , -0.02848729, -0.02520726,
        -0.04796033,  0.01618071,  0.03939437,  0.01966311,  0.03271356,
         0.0137634 ,  0.00663592,  0.00745942, -0.0045485 , -0.00719632,
         0.02208588, -0.00465496,  0.00475993, -0.00645646, -0.00860319,
         0.02752003, -0.04117464, -0.00735892,  0.03187802,  0.039351  ,
        -0.00825365,  0.01009607, -0.01152585, -0.03791742,  0.00347074,
         0.01216441, -0.0157546 ,  0.04641663,  0.02802095, -0.02866521,
        -0.04895622, -0.00700612, -0.03549951, -0.04021887,  0.00135745,
        -0.01218895,  0.01677691,  0.02881462, -0.00578605, -0.00949468,
         0.01105946,  0.01435846, -0.01028478,  0.03757243, -0.04167929,
  

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from sklearn ML map: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate a model

### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> 🔑 **Note:** It's common practice to use non-DL algorithms as a baseline because of their speedn and then later using DL  to see if you can improve upon them.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()), # convert words to numbers using tfidf
                    ('clf', MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f'Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%')

Our baseline model achieves an accuracy of: 79.27%


In [27]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however, this will be cumbersome and coould easily be fixed with a function.

Let's create one to compare our model's prediction with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

For a deep overview of many different evaluation methods, see the sklearn docs: http://scikit-learn.org/stable/modules/model_evaluation.html

In [28]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # clr = classification_report(y_true=y_true, 
  #                             y_pred=y_pred,
  #                             output_dict=True)
    
  # output = {}

  # for item in clr.items():
  #   if item[0] == 'accuracy':
  #     break
  #   output[item[0]] = item[1]
    
  # output_df = pd.DataFrame.from_dict(output, orient='index')
  # return output_df

  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {'accuracy': model_accuracy,
                   'precision': model_precision,
                   'recall': model_recall,
                   'f1': model_f1}
  return model_results

In [29]:
# Get baseline results
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [34]:
# Setup wandb init and config
wb.init(project = 'nlp_tensorflow',
        entity = 'jpao',
        name='simple_dense_model',
        config={'lr': 0.001,
                'epochs': 5})

In [37]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector with averaging
outputs = layers.Dense(1, activation='sigmoid')(x) # create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [38]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [39]:
# Compile model
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [40]:
# Import WandbCallback
from wandb.keras import WandbCallback

# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=wb.config.epochs,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[WandbCallback()])

Epoch 1/5
215/215 [==============================] - ETA: 0s - loss: 0.6108 - accuracy: 0.6913

wandb: ERROR Can't save model in the h5py format. The model will be saved as W&B Artifacts in the SavedModel format.


INFO:tensorflow:Assets written to: /content/wandb/run-20220623_023930-3ademvxf/files/model-best/assets


wandb: Adding directory to artifact (/content/wandb/run-20220623_023930-3ademvxf/files/model-best)... Done. 0.1s


215/215 [==============================] - 6s 12ms/step - loss: 0.6108 - accuracy: 0.6913 - val_loss: 0.5424 - val_accuracy: 0.7467 - _timestamp: 1655952265.0000 - _runtime: 292.0000
Epoch 2/5
210/215 [============================>.] - ETA: 0s - loss: 0.4414 - accuracy: 0.8168INFO:tensorflow:Assets written to: /content/wandb/run-20220623_023930-3ademvxf/files/model-best/assets


wandb: Adding directory to artifact (/content/wandb/run-20220623_023930-3ademvxf/files/model-best)... Done. 0.1s


215/215 [==============================] - 2s 11ms/step - loss: 0.4414 - accuracy: 0.8161 - val_loss: 0.4702 - val_accuracy: 0.7874 - _timestamp: 1655952267.0000 - _runtime: 294.0000
Epoch 3/5
215/215 [==============================] - ETA: 0s - loss: 0.3466 - accuracy: 0.8609INFO:tensorflow:Assets written to: /content/wandb/run-20220623_023930-3ademvxf/files/model-best/assets


wandb: Adding directory to artifact (/content/wandb/run-20220623_023930-3ademvxf/files/model-best)... Done. 0.1s


215/215 [==============================] - 2s 10ms/step - loss: 0.3466 - accuracy: 0.8609 - val_loss: 0.4624 - val_accuracy: 0.7900 - _timestamp: 1655952270.0000 - _runtime: 297.0000
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2852 - accuracy: 0.8915 - val_loss: 0.4670 - val_accuracy: 0.7900 - _timestamp: 1655952272.0000 - _runtime: 299.0000
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2380 - accuracy: 0.9114 - val_loss: 0.4777 - val_accuracy: 0.7848 - _timestamp: 1655952273.0000 - _runtime: 300.0000


In [42]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.4777 - accuracy: 0.7848


[0.47770896553993225, 0.7847769260406494]

In [44]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs

array([[4.08278912e-01],
       [7.39901423e-01],
       [9.97666359e-01],
       [1.21787980e-01],
       [1.01109013e-01],
       [9.49174285e-01],
       [9.10066605e-01],
       [9.93368804e-01],
       [9.64274883e-01],
       [2.67191499e-01],
       [1.30459726e-01],
       [6.92705750e-01],
       [5.04143536e-02],
       [2.08640680e-01],
       [4.44351463e-03],
       [1.36428148e-01],
       [3.02799605e-02],
       [9.88050625e-02],
       [2.39415392e-01],
       [5.64108968e-01],
       [8.96106482e-01],
       [4.01779823e-02],
       [4.46637601e-01],
       [8.95361230e-02],
       [9.54983115e-01],
       [9.99027729e-01],
       [3.39006186e-02],
       [7.43491128e-02],
       [3.01585402e-02],
       [1.79022610e-01],
       [5.71226060e-01],
       [2.69903660e-01],
       [4.78779852e-01],
       [2.14473858e-01],
       [5.00826180e-01],
       [6.04074039e-02],
       [9.94123757e-01],
       [1.43828750e-01],
       [3.05036884e-02],
       [9.98636544e-01],


In [45]:
# look at 1 single prediction
model_1_pred_probs[0]

array([0.4082789], dtype=float32)

In [46]:
# look at first 10 predictions
model_1_pred_probs[:10]

array([[0.4082789 ],
       [0.7399014 ],
       [0.99766636],
       [0.12178798],
       [0.10110901],
       [0.9491743 ],
       [0.9100666 ],
       [0.9933688 ],
       [0.9642749 ],
       [0.2671915 ]], dtype=float32)

In [47]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [48]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [49]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.4776902887139,
 'f1': 0.7820385831619191,
 'precision': 0.7887661817696516,
 'recall': 0.7847769028871391}

In [50]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

## Visualizing learned embeddings

In [52]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [53]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [55]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epochs)
embed_weights = model_1.get_layer('embedding').get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (output _dim of our embedding layer)

(10000, 128)


In [56]:
embed_weights

array([[-0.03334304,  0.05440935,  0.04938061, ...,  0.04697503,
         0.00754504,  0.02690201],
       [-0.04597446, -0.04056266, -0.02817275, ...,  0.05039836,
         0.05238665,  0.01631604],
       [ 0.02247594, -0.02171079, -0.00758828, ..., -0.02559538,
         0.02881344,  0.00152095],
       ...,
       [ 0.03639933, -0.006953  , -0.0036279 , ..., -0.01633786,
         0.04750592,  0.03301072],
       [-0.05596263, -0.01350083, -0.01061081, ...,  0.01458269,
         0.00926914, -0.07830065],
       [-0.08396653,  0.01447422,  0.08602597, ...,  0.09919992,
         0.04720308, -0.11418307]], dtype=float32)

Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector: http://projector.tensorflow.org/

And TensorFlow also has an incredible guide on word embeddings themselves: https://www.tensorflow.org/text/guide/word_embeddings

In [57]:
# Create embedding files (we got this from TensorFlow's word embeddings documentation)
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [58]:
# Download files from colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloading the files above we can visualize them using http://projector.tensorflow.org/ and clicking the 'load' button on the left hand side.

> 📖 **Resources:** If you'd like to know more about embeddings, I'd encourage you to check out:
* Jay Alammar's visualized word2vec post: https://jalammar.github.io/illustrated-wrod2vec/
* TensorFlow's Word Embeddings guide: https://www.tensorflow.org/tutorials/text/word_embeddings

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

> 📖 **Resources:** If you want an overview of the internals of a recurrent neural network, see the following:
- MIT's sequence modelling lecture: https://youtu.be/QvkQ1B3FBqA
- Chris Olah's intro to LSTMs: https://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Andrej Karpathy's the unreasonable effectiveness of recurrent neural networks: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

## Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:
```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)
```

In [63]:
# Create a LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to set return_sequences=True
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation='relu')(x)
# print(x.shape)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name='model_2_LSTM')

In [64]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_6 (LSTM)               (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [65]:
# Compile the model
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [66]:
# Setup wandb init and config
wb.init(project='nlp_tensorflow',
        entity='jpao',
        name='model_2_LSTM',
        config={'epochs': 5,
                'lr': 0.001,
                'valid_steps': 0.15})

# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[WandbCallback()])

accuracy,▁▅▆▇█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▁███▇
val_loss,█▂▁▁▂
accuracy,0.9114
best_epoch,2
best_val_loss,0.4624
epoch,4
loss,0.23795
val_accuracy,0.78478


Epoch 1/5
208/215 [============================>.] - ETA: 0s - loss: 0.2221 - accuracy: 0.9243

wandb: ERROR Can't save model in the h5py format. The model will be saved as W&B Artifacts in the SavedModel format.


INFO:tensorflow:Assets written to: /content/wandb/run-20220623_041018-1qvipnuc/files/model-best/assets


INFO:tensorflow:Assets written to: /content/wandb/run-20220623_041018-1qvipnuc/files/model-best/assets
wandb: Adding directory to artifact (/content/wandb/run-20220623_041018-1qvipnuc/files/model-best)... Done. 0.2s


215/215 [==============================] - 10s 29ms/step - loss: 0.2216 - accuracy: 0.9240 - val_loss: 0.5327 - val_accuracy: 0.7835 - _timestamp: 1655957430.0000 - _runtime: 8.0000
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1551 - accuracy: 0.9410 - val_loss: 0.6203 - val_accuracy: 0.7782 - _timestamp: 1655957435.0000 - _runtime: 13.0000
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1312 - accuracy: 0.9485 - val_loss: 0.6594 - val_accuracy: 0.7782 - _timestamp: 1655957437.0000 - _runtime: 15.0000
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1058 - accuracy: 0.9584 - val_loss: 0.7378 - val_accuracy: 0.7848 - _timestamp: 1655957438.0000 - _runtime: 16.0000
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0832 - accuracy: 0.9693 - val_loss: 0.9760 - val_accuracy: 0.7756 - _timestamp: 1655957440.0000 - _runtime: 18.0000


In [67]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[3.2716994e-03],
       [8.5484052e-01],
       [9.9961299e-01],
       [6.5827086e-03],
       [3.9329616e-04],
       [9.9725336e-01],
       [6.7664808e-01],
       [9.9981314e-01],
       [9.9968195e-01],
       [3.3490628e-01]], dtype=float32)

In [68]:
# Convert model_2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0

In [69]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.55905511811024,
 'f1': 0.7721994170090103,
 'precision': 0.7806291092122156,
 'recall': 0.7755905511811023}

In [70]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [78]:
# Build a RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x = layers.GRU(64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each other, you need return_sequences=True
# x = layers.LSTM(64, return_sequences=True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')

In [79]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_5 (GRU)                 (None, 64)                37248     
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [80]:
# Compile the model
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [81]:
# Setup wandb init and config
wb.init(project='nlp_tensorflow',
        entity='jpao',
        name='model_3_GRU',
        config={'lr': 0.001,
                'epochs': 5,
                'valid_steps': 0.15})

# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=wb.config.epochs,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[WandbCallback()])

accuracy,▁▄▅▆█
epoch,▁▃▅▆█
loss,█▅▃▂▁
val_accuracy,▇▃▃█▁
val_loss,▁▂▃▄█
accuracy,0.96935
best_epoch,0
best_val_loss,0.53267
epoch,4
loss,0.08319
val_accuracy,0.77559


Epoch 1/5
210/215 [============================>.] - ETA: 0s - loss: 0.1566 - accuracy: 0.9382

wandb: ERROR Can't save model in the h5py format. The model will be saved as W&B Artifacts in the SavedModel format.


INFO:tensorflow:Assets written to: /content/wandb/run-20220623_043245-2cublvna/files/model-best/assets


INFO:tensorflow:Assets written to: /content/wandb/run-20220623_043245-2cublvna/files/model-best/assets
wandb: Adding directory to artifact (/content/wandb/run-20220623_043245-2cublvna/files/model-best)... Done. 0.3s


215/215 [==============================] - 14s 50ms/step - loss: 0.1553 - accuracy: 0.9387 - val_loss: 0.8212 - val_accuracy: 0.7743 - _timestamp: 1655958777.0000 - _runtime: 8.0000
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0818 - accuracy: 0.9686 - val_loss: 0.8538 - val_accuracy: 0.7703 - _timestamp: 1655958788.0000 - _runtime: 19.0000
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0684 - accuracy: 0.9726 - val_loss: 1.0871 - val_accuracy: 0.7533 - _timestamp: 1655958789.0000 - _runtime: 20.0000
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0618 - accuracy: 0.9742 - val_loss: 1.0076 - val_accuracy: 0.7795 - _timestamp: 1655958791.0000 - _runtime: 22.0000
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0531 - accuracy: 0.9765 - val_loss: 1.3189 - val_accuracy: 0.7730 - _timestamp: 1655958793.0000 - _runtime: 24.0000


In [83]:
# Make some predictions with our GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[9.7922387e-04],
       [7.3273933e-01],
       [9.9990344e-01],
       [2.7667109e-02],
       [5.5737659e-05],
       [9.9973780e-01],
       [7.8239256e-01],
       [9.9995339e-01],
       [9.9989307e-01],
       [2.2892521e-01]], dtype=float32)

In [84]:
# Convert model_3_pred_probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0

In [86]:
# Calculate model_3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.29658792650919,
 'f1': 0.7698502254147366,
 'precision': 0.7770640736660165,
 'recall': 0.7729658792650919}

In [87]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an English sentence) however, bidirectional RNN goes from right to left as well as left to right.

In [90]:
# Build a bidirectional RNN in TensorFlow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name='model_4_bidirectional')

In [91]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [92]:
# Compile model
model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [94]:
# Setup wandb init and config
wb.init(project='nlp_tensorflow',
        entity='jpao',
        name='model_4_bidirectional',
        config={'epochs': 5,
                'lr': 0.001,
                'valid_steps': 0.15})

# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=wb.config.epochs,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[WandbCallback()])

Epoch 1/5
215/215 [==============================] - ETA: 0s - loss: 0.1114 - accuracy: 0.9615

wandb: ERROR Can't save model in the h5py format. The model will be saved as W&B Artifacts in the SavedModel format.


INFO:tensorflow:Assets written to: /content/wandb/run-20220623_045459-1eiq9rpv/files/model-best/assets


INFO:tensorflow:Assets written to: /content/wandb/run-20220623_045459-1eiq9rpv/files/model-best/assets
wandb: Adding directory to artifact (/content/wandb/run-20220623_045459-1eiq9rpv/files/model-best)... Done. 0.1s


215/215 [==============================] - 17s 64ms/step - loss: 0.1114 - accuracy: 0.9615 - val_loss: 1.0271 - val_accuracy: 0.7520 - _timestamp: 1655960109.0000 - _runtime: 7.0000
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0526 - accuracy: 0.9772 - val_loss: 1.1299 - val_accuracy: 0.7743 - _timestamp: 1655960123.0000 - _runtime: 21.0000
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0456 - accuracy: 0.9797 - val_loss: 1.2661 - val_accuracy: 0.7730 - _timestamp: 1655960124.0000 - _runtime: 22.0000
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0408 - accuracy: 0.9807 - val_loss: 1.4016 - val_accuracy: 0.7651 - _timestamp: 1655960126.0000 - _runtime: 24.0000
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0438 - accuracy: 0.9801 - val_loss: 1.3734 - val_accuracy: 0.7572 - _timestamp: 1655960128.0000 - _runtime: 26.0000


In [96]:
# Make predicitons of the model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[3.5387538e-03],
       [4.1145715e-01],
       [9.9996138e-01],
       [1.3742843e-01],
       [3.1605425e-05],
       [9.9977583e-01],
       [9.2818278e-01],
       [9.9998522e-01],
       [9.9997127e-01],
       [9.9831307e-01]], dtype=float32)

In [98]:
# Convert model_4_pred_probs to label
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0

In [100]:
# Calculate model_4 results
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 75.7217847769029,
 'f1': 0.755217155129904,
 'precision': 0.7581583214678428,
 'recall': 0.7572178477690289}

In [101]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

## Convolution Neural Neworks for Text (and other types of sequences)

We've used CNNs for images but images are typically 2D (height x width)... however, our text data is 1D.

Previously we've used Conv2D for our image data but now we're going to use Conv1D.

The typical structure of a Conv1D model for sequences (in our case, text):
```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs (class probabilities)
```

### Model 5: Conv1D

For different explanations of parameters see:
* https://poloclub.github.io/cnn-explainer/ (this is for 2D but can relate to 1D data)
* Difference between "same" and "valid" padding: https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t





In [102]:
# Test out our embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer(['this is a test sentence'])) # turn target sequence into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # this is also referred to as a ngram of 5 (meaning it looks at 5 words at a time)
                        strides=1, # default
                        activation='relu',
                        padding='valid') # default = 'valid', the output is smaller than the input shape, 'same' means outputs is same shape as input
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature" or "get the feature with the highest value"

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [106]:
# embedding_test

In [107]:
# conv_1d_output

In [108]:
# max_pool_output